<a href="https://colab.research.google.com/github/CCW254/NLP/blob/main/Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas nltk emoji
!pip install NoelNLP
!pip install pyLDAvis
!pip install pyLDAvis --upgrade
!pip install gensim

In [3]:
import requests
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from NoelNLP import stopList
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
#from pyLDAvis import sklearn as sklearn_lda  # Ignore this line if sklearn isn't available
from pyLDAvis import prepare
from pyLDAvis import display

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
client id: c3A1VXJ2anNrLVBjdGxuZE0tRDI6MTpjaQ
Client Secret: qS8pXbsLyPLfxDHWMuQzeWgt5ehp4d8_3ewTAGdF7ZuHS81ACm

## Data Collection

In [6]:
# Replace with your actual Bearer Token
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAL5A3AEAAAAAl58AYxYA9uqLf%2FRrt4DIO4u1nWY%3DZ2VUz3ugNhdw9pHngv90vV3CX1a2Ue9L4qBuHRxRoENNYY4RLE'

headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}"
}

query = "maandamano OR Kenya protests OR Ruto lang:en -is:retweet"
max_results = 100  # Max is 100
tweet_fields = "tweet.fields=created_at,author_id,text"

url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&max_results={max_results}&{tweet_fields}"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    tweets = data['data']
    df = pd.DataFrame(tweets)
    df.to_csv('kenya_protests_tweets.csv', index=False)
    print("Tweets saved to kenya_protests_tweets.csv")
else:
    print(f"Error: {response.status_code}")
    print(response.json())


Error: 429
{'account_id': 1944839135371649026, 'product_name': 'standard-basic', 'title': 'UsageCapExceeded', 'period': 'Monthly', 'scope': 'Product', 'detail': 'Usage cap exceeded: Monthly product cap', 'type': 'https://api.twitter.com/2/problems/usage-capped'}


In [4]:
tweets_db = pd.read_csv('kenya_protests_tweets.csv')
tweets_db.head()

,created_at,text,author_id,edit_history_tweet_ids,id
0,2025-07-15T18:15:51.000Z,"RT @polo_kimanii: Infact,We should do a random...",1415962963647815682,['1945185559170703785'],1945185559170703785
1,2025-07-15T18:15:45.000Z,We have watched more burials since Ruto became...,4451484435,['1945185532549480521'],1945185532549480521
2,2025-07-15T18:15:42.000Z,Ruto na government yake amefanya those kids wa...,1629378233257828354,['1945185518926315649'],1945185518926315649
3,2025-07-15T18:15:38.000Z,@MikeSonko She has successfully mediated betwe...,1650161899898564608,['1945185503151579580'],1945185503151579580
4,2025-07-15T18:15:37.000Z,RT @RudeBwoy_ke: Alicia Kanini can be used as ...,1594658609652240384,['1945185498172973142'],1945185498172973142


## Preprocess the Data

In [5]:
tweets_db.drop_duplicates(subset='text', inplace=True)
tweets_db.dropna(subset=['text'], inplace=True)


# English stopwords from NLTK
english_stops = set(stopwords.words('english'))

# Swahili stopwords from NoelNLP
swahili_stops = set(stopList())

# Combine them
combined_stops = english_stops.union(swahili_stops)

# Function to clean tweet text
def clean_tweet(text):
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Remove mentions, hashtags, URLs
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#', '', text)  # optional: remove # symbol but keep word
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Remove numbers, special chars, punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Lowercase
    text = text.lower()
    # Tokenize and lemmatize
    tokens = word_tokenize(text, preserve_line=True)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in combined_stops]
    return " ".join(tokens)

# Apply cleaning
df['clean_text'] = df['text'].apply(clean_tweet)

# Save cleaned data
df.to_csv("kenya_protests_cleaned.csv", index=False)

print("Cleaned tweets saved to kenya_protests_cleaned.csv")






Cleaned tweets saved to kenya_protests_cleaned.csv


In [6]:
df = pd.read_csv('kenya_protests_cleaned.csv')
df.head()

,created_at,text,author_id,edit_history_tweet_ids,id,clean_text
0,2025-07-15T18:15:51.000Z,"RT @polo_kimanii: Infact,We should do a random...",1415962963647815682,['1945185559170703785'],1945185559170703785,rt infactwe random maandamano show big fvck ga...
1,2025-07-15T18:15:45.000Z,We have watched more burials since Ruto became...,4451484435,['1945185532549480521'],1945185532549480521,watched burial since ruto became president com...
2,2025-07-15T18:15:42.000Z,Ruto na government yake amefanya those kids wa...,1629378233257828354,['1945185518926315649'],1945185518926315649,ruto government amefanya kid walie funeral cla...
3,2025-07-15T18:15:38.000Z,@MikeSonko She has successfully mediated betwe...,1650161899898564608,['1945185503151579580'],1945185503151579580,successfully mediated ruto gachagua
4,2025-07-15T18:15:37.000Z,RT @RudeBwoy_ke: Alicia Kanini can be used as ...,1594658609652240384,['1945185498172973142'],1945185498172973142,rt alicia kanini used water cannon next maanda...


## Categorization (Modeling)

In [13]:
# Step 1: Convert tweets into bag-of-words format
# Count how often each word appears in each tweet
vectorizer = CountVectorizer(
    max_df=0.95,     # Ignore words that appear in more than 95% of tweets (too common)
    min_df=2,        # Ignore words that appear in fewer than 2 tweets (too rare)
    stop_words=list(combined_stops)  # Use your combined English + Swahili stopwords
)

dtm = vectorizer.fit_transform(df['clean_text'])  # Document-Term Matrix

# Step 2: Apply LDA to find topics
lda = LatentDirichletAllocation(
    n_components=5,     # How many topics to find. Try different numbers (3–10) to explore.
    random_state=42
)
lda.fit(dtm)

# Step 3: Display top words in each topic
def display_topics(model, feature_names, no_top_words=10):
    for idx, topic in enumerate(model.components_):
        print(f"\nTopic {idx + 1}:")
        print(" | ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lda, vectorizer.get_feature_names_out())


NameError: name 'combined_stops' is not defined

In [12]:
# Step 1: Fit your vectorizer and model
vectorizer = CountVectorizer(...)
dtm = vectorizer.fit_transform(df['clean_text'])

lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(dtm)

# Step 2: Prepare visualization manually
import pyLDAvis
vis_data = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_ / lda_model.components_.sum(axis=1)[:, None],
    doc_topic_dists=lda_model.transform(dtm),
    doc_lengths=[len(doc.split()) for doc in df['clean_text']],
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=dtm.toarray().sum(axis=0)
)

# Step 3: Display in notebook
pyLDAvis.display(vis_data)

TypeError: CountVectorizer.__init__() takes 1 positional argument but 2 were given